<style>div.container { width: 100% }</style>
<img style="float:left;  vertical-align:text-bottom;" height="65" width="172" src="../assets/PyViz_logo_wm_line.png" />
<div style="float:right; vertical-align:text-bottom;"><h2>Tutorial 06. Network Graphs</h2></div>

Visualizing and working with network graphs is a common problem in many different disciplines. HoloViews provides the ability to represent and visualize graphs very simply and easily with facilities for interactively exploring the nodes and edges of the graph, especially using the Bokeh plotting interface. It can also make use of Datashader for plotting large graphs, and NetworkX for some convenient graph functions:

<div style="margin: 10px">
<a href="http://holoviews.org"><img style="margin:8px; display:inline; object-fit:scale-down; max-height:150px" src="../assets/holoviews.png"/></a>
<a href="http://bokeh.pydata.org"><img style="margin:8px; display:inline; object-fit:scale-down; max-height:150px" src="../assets/bokeh.png"/></a>
<a href="http://numpy.org"><img style="margin:8px; display:inline; object-fit:scale-down; max-height:150px" src="../assets/numpy.png"/></a>
<a href="http://pandas.pydata.org"><img style="margin:8px; display:inline; object-fit:scale-down; max-height:140px" src="../assets/pandas.png"/></a>
<a href="http://networkx.github.io"><img style="margin:8px; display:inline; object-fit:scale-down; max-height:140px" src="../assets/networkx.png"/></a>
</div>

In [ ]:
import numpy as np
import pandas as pd
import holoviews as hv
from holoviews import opts, dim
import networkx as nx

hv.extension('bokeh')

defaults = dict(width=400, height=400, padding=0.1)
hv.opts.defaults(
    opts.EdgePaths(**defaults), opts.Graph(**defaults), opts.Nodes(**defaults))

The HoloViews ``Graph`` ``Element`` differs from other elements in HoloViews in that it consists of multiple sub-elements. The ``Graph`` element itself holds the data that indicates whether each node is connected to each other node. By default the element will automatically compute concrete ``x`` and ``y`` positions for the nodes and represent them using a ``Nodes`` element, which is stored on the Graph. The abstract edges and concrete node positions are sufficient to render the ``Graph`` by drawing straight-line edges between the nodes. In order to supply explicit edge paths we can also declare ``EdgePaths``, providing explicit coordinates for each edge to follow.

To summarize, a ``Graph`` consists of three different components:

* The ``Graph`` itself holds the abstract edges stored as a table of node index pairs.
* The ``Nodes`` hold the concrete ``x`` and ``y`` positions of each node along with a node ``index``. The ``Nodes`` may also define any number of value dimensions, which can be revealed when hovering over the nodes or to color the nodes by.
* The ``EdgePaths`` can optionally be supplied to declare explicit node paths.

#### A simple Graph

Let's start by declaring a very simple graph connecting one node to all others. If we simply supply the abstract connectivity of the ``Graph``, it will automatically compute a layout for the nodes using the ``layout_nodes`` operation, which defaults to a circular layout:

In [ ]:
# Declare abstract edges
N = 8
node_indices = np.arange(N)
source = np.zeros(N)
target = node_indices

padding = dict(x=(-1.2, 1.2), y=(-1.2, 1.2))

simple_graph = hv.Graph(((source, target),)).redim.range(**padding)
simple_graph

#### Accessing the nodes and edges

We can easily access the ``Nodes`` and ``EdgePaths`` on the ``Graph`` element using the corresponding properties:

In [ ]:
simple_graph.nodes + simple_graph.edgepaths

#### Supplying explicit paths

Next we will extend this example by supplying explicit edges:

In [ ]:
def bezier(start, end, control, steps=np.linspace(0, 1, 100)):
    return (1-steps)**2*start + 2*(1-steps)*steps*control+steps**2*end

x, y = simple_graph.nodes.array([0, 1]).T

paths = []
for node_index in node_indices:
    ex, ey = x[node_index], y[node_index]
    paths.append(np.column_stack([bezier(x[0], ex, 0), bezier(y[0], ey, 0)]))
    
bezier_graph = hv.Graph(((source, target), (x, y, node_indices), paths)).redim.range(**padding)
bezier_graph

## Interactive features

#### Hover and selection policies

Thanks to Bokeh we can reveal more about the graph by hovering over the nodes and edges. The ``Graph`` element provides an ``inspection_policy`` and a ``selection_policy``, which define whether hovering and selection highlight edges associated with the selected node or nodes associated with the selected edge. These policies can be toggled by setting the policy to ``'nodes'`` (the default) or ``'edges'``.

In [ ]:
bezier_graph.options(inspection_policy='edges')

In addition to changing the policy, we can also change the colors used when hovering and selecting nodes:

In [ ]:
bezier_graph.opts(
    opts.Graph(inspection_policy='nodes', tools=['hover', 'box_select'],
               edge_hover_line_color='green', node_hover_fill_color='red'))

#### Additional information


We can also associate additional information with the nodes and edges of a graph. By constructing the ``Nodes`` explicitly we can declare additional value dimensions, which are revealed when hovering and/or can be mapped to the color by setting the ``color`` to the dimension name ('Weight'). We can also associate additional information with each edge by supplying a value dimension to the ``Graph`` itself, which we can map to various style options, e.g. by setting the ``edge_color`` and ``edge_line_width``.

In [ ]:
node_labels = ['Output']+['Input']*(N-1)
np.random.seed(7)
edge_labels = np.random.rand(8)

nodes = hv.Nodes((x, y, node_indices, node_labels), vdims='Type')
graph = hv.Graph(((source, target, edge_labels), nodes, paths), vdims='Weight')

(graph + graph.opts(inspection_policy='edges', clone=True)).opts(
    opts.Graph(node_color='Type', edge_color='Weight', cmap='Set1',
               edge_cmap='viridis', edge_line_width=hv.dim('Weight')*10))

If you want to supply additional node information without speciying explicit node positions you may pass in a ``Dataset`` object consisting only of various value dimensions.

In [ ]:
node_info = hv.Dataset(node_labels, vdims='Label')
hv.Graph(((source, target), node_info)).opts(node_color='Label', cmap='Set1')

## Working with NetworkX

NetworkX is a very useful library when working with network graphs, and the Graph Element provides ways of importing a NetworkX Graph directly. Here we will load the Karate Club graph and use the ``circular_layout`` function provided by NetworkX to lay it out:

In [ ]:
G = nx.karate_club_graph()
karate_club_opts = opts.Graph(tools=['hover'], node_color=dim('club'), cmap='Set1')
hv.Graph.from_networkx(G, nx.layout.circular_layout).opts(karate_club_opts)

#### Animating graphs

Like all other elements ``Graph`` can be updated in a ``HoloMap`` or ``DynamicMap``. Here we animate how the Fruchterman-Reingold force-directed algorithm lays out the nodes in real time.

In [ ]:
G = nx.karate_club_graph()

def get_graph(iteration):
    np.random.seed(10)
    return hv.Graph.from_networkx(G, nx.spring_layout, iterations=iteration)

hv.HoloMap({i: get_graph(i).opts(karate_club_opts) for i in range(5, 30, 5)}, kdims='Iterations')

## Real world graphs

As a final example let's look at a slightly larger graph. We will load a dataset of a Facebook network consisting a number of friendship groups identified by their ``'circle'``. We will load the edge and node data using pandas and then color each node by their friendship group using many of the things we learned above.

In [ ]:
kwargs = dict(width=800, height=800, xaxis=None, yaxis=None)
opts.defaults(opts.Nodes(**kwargs), opts.Graph(**kwargs))

colors = ['#000000']+hv.Cycle('Category20').values
edges_df = pd.read_csv('../data/fb_edges.csv')
fb_nodes = hv.Nodes(pd.read_csv('../data/fb_nodes.csv')).sort()
fb_graph = hv.Graph((edges_df, fb_nodes), label='Facebook Circles')

fb_graph.opts(cmap=colors, node_size=10, edge_line_width=1,
              node_line_color='gray', node_color='circle')

## Bundling graphs

Later, in [Working with Large Datasets](10_Working_with_Large_Datasets.ipynb) we will see how the [Datashader](http://datashader.org/) library allows us to render very large datasets efficiently. In this section, we use the algorithms for bundling the edges of large graphs that are available in datashader via HoloViews.

In [ ]:
from holoviews.operation.datashader import datashade, bundle_graph
bundled = bundle_graph(fb_graph)
bundled

## Datashading graphs

For graphs with a large number of edges we can datashade the paths and display the nodes separately. This loses some of the interactive features but will let you visualize quite large graphs. If the number of edges is much greater than the number of nodes, using datashader to render the edges still lets you interact with each node for hovering, even though the connections are now drawn as an image:

In [ ]:
(datashade(bundled, normalization='linear', width=800, height=800) * bundled.nodes).opts(
    opts.Nodes(color='circle', size=10, width=1000, cmap=colors, legend_position='right'))

### Applying selections

Alternatively we can select the nodes and edges by an attribute that resides on either. In this case we will select the nodes and edges for a particular circle and then overlay just the selected part of the graph on the datashaded plot. Note that selections on the ``Graph`` itself will select all nodes that connect to one of the selected nodes. In this way a smaller subgraph can be highlighted and the larger graph can be datashaded to reduce the file size.

In [ ]:
datashaded = datashade(bundle_graph(fb_graph), normalization='linear', width=800, height=800)
selected = bundled.select(circle='circle15').opts(node_fill_color='white')
datashaded * selected

To select just the nodes that are in 'circle15' set the ``selection_mode='nodes'`` overriding the default of 'edges':

In [ ]:
bundled.select(circle='circle15', selection_mode='nodes')

# Onwards

Having seen how to visualize and interactively explore graphical data, we now go on to demonstrate how to visualize and explore a specific domain: [Geographic Data](./07_Geographic_Data.ipynb). While domain specific, geographic data is both very common and typically awkward to handle.